In [1]:

import os 
import redis
import sys  
sys.path.append('..')
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
redis_client = redis.Redis(
    host=os.getenv('REDIS_HOST',""),
    port=os.getenv('REDIS_PORT',""),
)
redis_client.ping()

True

In [6]:
import redis

def get_redis_content_scan(host='localhost', port=6379, db=0, pattern='*', count=100):
    """
    Connects to Redis and lists content using the SCAN command, which is
    non-blocking and suitable for large datasets.

    Args:
        host (str): Redis host.
        port (int): Redis port.
        db (int): Redis database number.
        pattern (str): Pattern to match keys (e.g., 'user:*', '*').
                       Default is '*' for all keys.
        count (int): Hint for the number of elements to return per call to SCAN.
                     Redis may return more or fewer.

    Returns:
        dict: A dictionary where keys are Redis keys and values are
              dictionaries containing 'type' and 'value' of the Redis entry.
    """
    try:
        r = redis.StrictRedis(host=host, port=port, db=db, decode_responses=True)

        content = {}
        cursor = '0' # Initial cursor for SCAN

        print(f"Scanning Redis for keys matching pattern '{pattern}'...")

        while cursor != 0:
            cursor, keys = r.scan(cursor=cursor, match=pattern, count=count)

            for key in keys:
                entry_type = r.type(key) # type() returns string when decode_responses=True
                entry_value = None

                if entry_type == 'string':
                    entry_value = r.get(key)
                elif entry_type == 'list':
                    entry_value = r.lrange(key, 0, -1)
                elif entry_type == 'set':
                    entry_value = list(r.smembers(key))
                elif entry_type == 'hash':
                    entry_value = r.hgetall(key)
                elif entry_type == 'zset':
                    # zrange returns (member, score) tuples if withscores=True
                    entry_value = r.zrange(key, 0, -1, withscores=True)
                else:
                    entry_value = f"Unsupported or unknown type: {entry_type}"

                content[key] = {
                    'type': entry_type,
                    'value': entry_value
                }
        print(f"Finished scanning. Total {len(content)} keys found.")
        return content

    except redis.exceptions.ConnectionError as e:
        print(f"Error: Could not connect to Redis at {host}:{port}. Please ensure Redis is running.")
        print(f"Details: {e}")
        return {}
    except Exception as e:
        print(f"An unexpected error occurred while querying Redis: {e}")
        return {}

if __name__ == "__main__":
    # --- Setup: Add some sample data to Redis ---
    r_client = redis.StrictRedis(host='localhost', port=6379, db=0, decode_responses=True)
    print("\n--- Querying all content using SCAN ---")
    all_redis_content = get_redis_content_scan()
    if all_redis_content:
        for key, data in all_redis_content.items():
            print(f"Key: '{key}', Type: {data['type']}, Value: {data['value']}")


--- Querying all content using SCAN ---
Scanning Redis for keys matching pattern '*'...
Finished scanning. Total 1 keys found.
Key: 'message_store:default', Type: list, Value: ['{"type": "ai", "data": {"content": "\\u2705 \\u0110\\u00e3 t\\u1ea1o \\u0111\\u1ec1 thi v\\u00e0 l\\u01b0u th\\u00e0nh file tong_hieu.docx", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}}', '{"type": "human", "data": {"content": "\\nContext information:\\n[{\'content\': \'..........................................................................................................................................\\\\n..........................................................................................................................................\\\\nB. PH\\u1ea6N VI\\u1ebeT: (10 \\u0111i\\u1ec3m)\\\\n\\u0110\\u1ec1 b\\u00e0i: Xung quanh em c\\u00f3 r\\u1ea5t nhi\\u1ec1u c\\u00e2y

In [9]:
all_redis_content.get("message_store")

In [15]:
all_redis_content['message_store:default']["value"]


['{"type": "ai", "data": {"content": "\\u2705 \\u0110\\u00e3 t\\u1ea1o \\u0111\\u1ec1 thi v\\u00e0 l\\u01b0u th\\u00e0nh file tong_hieu.docx", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}}',
 '{"type": "human", "data": {"content": "\\nContext information:\\n[{\'content\': \'..........................................................................................................................................\\\\n..........................................................................................................................................\\\\nB. PH\\u1ea6N VI\\u1ebeT: (10 \\u0111i\\u1ec3m)\\\\n\\u0110\\u1ec1 b\\u00e0i: Xung quanh em c\\u00f3 r\\u1ea5t nhi\\u1ec1u c\\u00e2y c\\u1ed1i, m\\u1ed7i  lo\\u00e0i c\\u00e2y c\\u00f3 l\\u1ee3i \\u00edch v\\u00e0 v\\u1ebb \\u0111\\u1eb9p ri\\u00eang, h\\u00e3y vi\\u1ebft b\\u00e0i v\\u0103n mi\\u00eau t\\u